📄 Proyecto de Extracción y Procesamiento de Normativas en Ciberseguridad

Este proyecto procesa 4 documentos normativos en formato PDF, estructurando su contenido y permitiendo búsquedas eficientes mediante procesamiento de lenguaje natural (NLP) y almacenamiento en ChromaDB para búsqueda semántica.
📚 Documentos Utilizados 📚

Los siguientes documentos fueron extraídos y procesados:

    GDPR 2016/679 - Reglamento General de Protección de Datos (Unión Europea)
    GDPR 2018/1725 - Protección de datos en instituciones de la UE
    ISO 27001:2022 - Norma internacional para la gestión de seguridad de la información
    NIST - Marco de seguridad cibernética del Instituto Nacional de Estándares y Tecnología (EE.UU.)

Estos documentos fueron procesados en varias etapas para convertirlos en un formato estructurado, aplicar técnicas de NLP y almacenar sus embeddings para búsqueda semántica.
📁 Estructura del Proyecto

📁 data/ → PDFs originales
📁 json/ → Archivos JSON extraídos de los PDFs
📁 processed/ → Archivos procesados con NLP y embeddings
📁 scripts/ → Código Python para procesamiento

In [ ]:
!pip install pymupdf
!pip install pytesseract
!pip install pdf2image
!apt-get install poppler-utils -y  # Necesario para pdf2image
!apt-get install tesseract-ocr-spa -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-spa is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [ ]:
import fitz  # PyMuPDF
import pytesseract  # OCR con Tesseract
from pdf2image import convert_from_path  # Convertir PDF a imágenes para OCR
import json  # Guardar el texto en formato JSON
import os  # Manejar rutas de archivos

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive_folder = "/content/drive/My Drive/Docs-Ciberseguridad/"

# Rutas de los PDFs en Google Drive
pdf_files = {
    "GDPR_2016_679": "/content/drive/My Drive/Docs-Ciberseguridad/GDPR 2016679.pdf",
    "GDPR_2018_1725": "/content/drive/My Drive/Docs-Ciberseguridad/GDPR 20181725.pdf",
    "ISO_27001_2022": "/content/drive/My Drive/Docs-Ciberseguridad/ISO-27001-2022.pdf",
    "NIST_CSF_2_0": "/content/drive/My Drive/Docs-Ciberseguridad/NIST.pdf"
}

In [ ]:
# Función para extraer texto digital y OCR si hay imágenes
def extract_text_to_json(pdf_path):
    """Extrae texto digital y OCR de imágenes en un PDF y lo estructura en JSON."""
    doc = fitz.open(pdf_path)
    structured_data = {"title": os.path.basename(pdf_path), "sections": []}

    for page_num in range(len(doc)):
        page = doc[page_num]

        # Extraer texto digital primero
        text = page.get_text("text").strip()

        # Verificar si la página tiene imágenes
        images = page.get_images(full=True)
        ocr_text = ""

        if images:
            # Convertir la página en imagen
            image = convert_from_path(pdf_path, first_page=page_num+1, last_page=page_num+1)[0]
            # Aplicar OCR con Tesseract
            ocr_text = pytesseract.image_to_string(image, lang="spa").strip()

        # Combinar texto digital y OCR (si existe)
        combined_text = text + "\n\n" + ocr_text if text and ocr_text else text or ocr_text

        structured_data["sections"].append({"page": page_num + 1, "content": combined_text})

    return structured_data

In [ ]:
# Procesar cada PDF y guardarlo en formato JSON
output_json_files = {}
for name, path in pdf_files.items():
    print(f"Procesando {name} en formato JSON...")
    extracted_json = extract_text_to_json(path)

    # Guardar archivo JSON en Google Drive
    output_path = os.path.join(drive_folder, f"{name}.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(extracted_json, f, ensure_ascii=False, indent=4)

    output_json_files[name] = output_path
    print(f" Archivo JSON guardado en: {output_path}")

print(" Extracción completada. Archivos JSON listos en Google Drive.")

Procesando GDPR_2016_679 en formato JSON...
 Archivo JSON guardado en: /content/drive/My Drive/Docs-Ciberseguridad/GDPR_2016_679.json
Procesando GDPR_2018_1725 en formato JSON...
 Archivo JSON guardado en: /content/drive/My Drive/Docs-Ciberseguridad/GDPR_2018_1725.json
Procesando ISO_27001_2022 en formato JSON...
 Archivo JSON guardado en: /content/drive/My Drive/Docs-Ciberseguridad/ISO_27001_2022.json
Procesando NIST_CSF_2_0 en formato JSON...
 Archivo JSON guardado en: /content/drive/My Drive/Docs-Ciberseguridad/NIST_CSF_2_0.json
 Extracción completada. Archivos JSON listos en Google Drive.
